In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Weekly Sales = 주간 판매량

**데이터 불러오고 확인******

In [ ]:
train = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
test = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip')

In [ ]:
display(train,test)
"""
store : 상점 번호
dept(department) : 부서(가게의 부서 ex)식료품점, 의류점....)
Date : 날짜
isHoliday : 공휴일인지 아닌지
Weekly_sales : 하루 판매량

->test set에 Weekly_sales가 없는걸 보니, 이걸 예측해야함
"""

In [ ]:
# 데이터 5개 지우기
# 1) drop , axis=0
# 2) 조건주기.

#1)
train=train.drop(train['Weekly_Sales'].sort_values()[:5].index,axis=0)

In [ ]:
#2)
train = train[train['Weekly_Sales']>=-1100] 

In [ ]:
# baseline잡고
# 제출
"""
점수를 어떻게 올릴지에 초점맞추기
"""

"""
모델 : Randomforest why? 범주형데이터가 많아서
날짜 칼럼 포함해서 학습시키기
"""

In [ ]:
train['Date'] = train['Date'].astype('datetime64')
train['Date']

In [ ]:
test['Date'] = test['Date'].astype('datetime64')
#test['Date']

In [ ]:
train['year'] = train['Date'].dt.year

In [ ]:
test['year'] = test['Date'].dt.year

In [ ]:
train['year']

In [ ]:
test['year']

In [ ]:
# groupby함수 -> 년도별로 군집화한 다음, 년도별의 평균판매량 확인,비교
train.groupby('year')['Weekly_Sales'].mean()  #년도별로, Weekly_Sales군집화 후, 평균찾기
#by = 어떤 칼럼 기준 그룹화?

#결과 : 2010,2011,2012의 값이 다 비슷 -> 학습하는데 별 도움이 안 될 것 같다? 우선, 충분한 양의 데이터가 있는지 확인. -> !!!충분한 양의 데이터가 있으면(많으면)!!!, !!!!!작은 차이도 큰차이다 !!!!!



In [ ]:
#충분한 양의 데이터가 있는가?? 확인
train['year'].value_counts() # 데이터 많네 -> 작은 차이도 큰차이!!!
# 년도가 지나갈수록, 판매량이 줄어드네.

In [ ]:
#sns.boxplot(train['year'],train['Weekly_Sales'])

In [ ]:
train['month'] = train['Date'].dt.month
train['month']

In [ ]:
test['month'] = test['Date'].dt.month
test['month']

In [ ]:
train['day'] = train['Date'].dt.day
test['day'] = test['Date'].dt.day

#display(train,test)

In [ ]:
# 주 단위로 나눔
# 1년을 52주로
train['week'] = train['Date'].dt.week
test['week'] = test['Date'].dt.week


In [ ]:
# 4주로 나누기 (각 월당)
train['weeknum']=np.ceil(train['day']/7).astype('int32')  #각 행에 대해서 연산을 해주구나.
test['weeknum']=np.ceil(test['day']/7).astype('int32')

#

In [ ]:
# weekday는 추가x ->아래서 확인해보면, 금요일 하나만 있음 왜냐하면 '주간 판매량 계산이기때문'
# train['weekday'] = train['Date'].dt.weekday
# test['weekday'] = test['Date'].dt.weekday

In [ ]:
# plt.figure(figsize=(16,10))
# sns.boxplot(train['day'],train['Weekly_Sales'],showfliers=False) 
# # day를 추가하면, month를 더 세세하게 나눌 수 있다. 
# # -> 월 초,말의 패턴이 다름
# # -> 특정 날짜에 판매가 많이 되는 경향이 보임->이날은 행사 or 그런게 있나봄 
# # -> 공휴일+날짜 => 공휴일을 더 세세하게 나눌 수 있다.

In [ ]:
# 'day'칼럼이 진짜 도움이 될까? 확인
#train.groupby('year')['Weekly_Sales'].mean()
train.groupby('day')['Weekly_Sales'].mean()

In [ ]:
train['day'].value_counts()

In [ ]:
## 새롭게 추가! ##
# 점수 올리는 방법!(date이용 말고도)
# ->데이터셋에 없는 칼럼 추가 + 원래 칼럼을 변형
# ->여기선 추가적인 csv파일 존재

In [ ]:
store = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv')
store
#스토어의 type,size에 따라 판매량 다르겠네

In [ ]:
train = pd.merge(train,store,on='Store',how='left')
# on : 어떤 칼럼을 기준으로 합칠 것인가 =>여기선 겹치는 칼럼이 Store니까, Store를 기준으로 합침
# how : 어떻게 합칠것인가-> train,store 보면 ,를 기준으로 왼쪽으로 합치겠다=train에다가 합치겠다.
# concat이랑 다르게, 좀 더 smart하게 칼럼에 따라 합침
test = pd.merge(test,store,on='Store',how='left')
train

In [ ]:
# plt.figure(figsize=(16,10))
# sns.boxplot(train['Type'],train['Weekly_Sales'],showfliers=False) # showfliers : outlier를 보여줄까, 말까
# # 결과 : A일때가 판매가 훨씬 많다.

# train.groupby('Type')['Weekly_Sales'].mean()

In [ ]:
train

In [ ]:
#train = train.replace({'A':1,'B':2,'C':3})
train['Type'] = train['Type'].replace({'A':1,'B':2,'C':3})
test['Type'] = test['Type'].replace({'A':1,'B':2,'C':3})


In [ ]:
train

In [ ]:
test

In [ ]:
# train['weekday'].unique() # 4하나만 있네
# # train['weekday']에 있는 독특한 값이 뭐뭐 있는지 확인

In [ ]:
#####################################################
##       데이터 확인(그래프로)                   ######
#####################################################

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,10))
sns.boxplot(train['month'],train['Weekly_Sales'],showfliers=False)
#outlier가 너무 많다. -> 해석 불가능 -> 조건을 달아줘야 한다. -> holiday없을때,로 outlier줄여줘야 한다. -> 이거는 확인하기 위함이기 때문에, 확인하기위한 dataFrame새로 만듦(train3) or 옵션(showfliers)사용!!
#50%분포도 너무 아래 -> 박스의 위치도 너무 아래.

# 결과
# 최댓값의 위치가 차이가 많이 남
# 달마다 판매하는 분포가 다름 
# 중앙값은 비슷하지만

In [ ]:
plt.figure(figsize=(16,10))
sns.boxplot(train['week'],train['Weekly_Sales'],showfliers=False)

In [ ]:
plt.figure(figsize=(16,10))
sns.boxplot(train['weeknum'],train['Weekly_Sales'],showfliers=False)

In [ ]:
# month를 주면 갑자기 등수가 팍 오름 why??
# holiday + month의 시너지.
# + 평가지표는 WMAE -> 공휴일에 가중치를 5나 줌->공휴일 잘 맞추면 점수 더 잘 오름
# 다음에 할 것 : 달에 공휴일이 여러개! 

In [ ]:
# 인덱스로 데이터 접근!!!
train.iloc[0]

In [ ]:
train.iloc[0:3]

In [ ]:
train['Weekly_Sales']<50000 
#train[0,1,2,..]이렇게 넣으면 error남.
#즉, train[train['Weekly_Sales']<50000]하는건, 결국 column을 주는 거다. 조건을 만족하는

In [ ]:
train3= train[train['Weekly_Sales']<50000] # 조건을 주면, 이렇게 인덱스를 볼 수 있다. 아닐땐, iloc을 써야한다.(인덱스별로 보기 위해선)
train3


In [ ]:
min(train['Weekly_Sales'])

In [ ]:
train['Weekly_Sales'].median() # .median() = pandas함수

In [ ]:
plt.figure(figsize=(16,10))
sns.boxplot(train3['month'],train3['Weekly_Sales'])
#음수인 애들이 있네 -> LOG씌우면 -됨.
"""
y값에 LOG씌우기 위한 방법
1) 음수를 버린다 -> but 최고의 솔루션이 아님
2) 평행이동 -> 음수인 애들의 최솟값을 구해서 평행이동 한다. -> 이것도 부작용이 있다? -> 평행이동하고 log를 씌우기 때문에, 위로 갈수록 값의 분포가 좁아지기 때문.
3) outlier만 날리고, 적당히 더하면서 패턴이 무너지는 것을 방지하겠다. 
! 정규화하면, 데이터의 분포가 완전히 달라지므로 쓰면 안된다!!!!!
"""

In [ ]:
train['Weekly_Sales'].sort_values()[:30] #sort_values = 오름차순. [:30]은 하위 30개만 보겠다.
# 의미있는 데이터 = 패턴의 반복
# -..98로 끝나는 패턴-> 왜인지는 모르겠으나, Walmart의 규정이 그런가봄 =>살려야한다.
# =>위로 미루어보았을 때, 데이터 하위 5개 지우는게 가장 효과가 좋더라.
# 따라서, 5개지우고, +1100해주고 log씌워 학습!

In [ ]:
train[train['Weekly_Sales']<0]

In [ ]:
plt.figure(figsize=(16,10))
sns.distplot(train['Weekly_Sales'])

In [ ]:
np.log(train['Weekly_Sales'])

In [ ]:
train

In [ ]:
test

In [ ]:
test2 = test.drop(['Date'],axis=1)
test2.head()

In [ ]:
train2 = train.drop(['Date','Weekly_Sales'],axis=1)
train2.head()

In [ ]:
#################################################
##        모델링 및 학습 시작                   ##
#################################################

In [ ]:
from sklearn.ensemble import RandomForestRegressor 

rf = RandomForestRegressor(n_jobs=-1,n_estimators=100) #n_estimator는 모델을 100개 만들어 앙상블하겠단 의미인듯

rf.fit(train2,np.log(train['Weekly_Sales']+1100)) #이 대회에선, np.log()가 안되네. infinity때문에

In [ ]:
ans = rf.predict(test2)

test['Weekly_Sales']= np.exp(ans)-1100
test.head()

In [ ]:
sub = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')
sub.head()

In [ ]:
sub['Weekly_Sales']=test['Weekly_Sales']
sub.head()

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
sub

In [ ]:
sub.to_csv('Walmart_baseline.csv',index=False)